In [1]:
import json
import pandas as pd

filename = 'test.jsonl'
savename = 'test.csv'

data = []
with open(filename) as f:
    for line in f:
        data.append(json.loads(line))

data2 = []
for i in data:
    data2.append(i['text'])

data2 = pd.DataFrame(data2, columns=['text'])

labels = []
for i in data:
    labels.append(i['label'])
data2['label'] = labels
data2.to_csv(savename, index=False)

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (3058682447.py, line 4)

In [3]:
import torch
import torch.nn as nn
from torch import Tensor
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [5]:
class SampleLoader(Dataset):
    def __init__(self, annotation_file):
        super().__init__()
        self.samples = pd.read_csv(annotation_file)
        
    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        txt = self.samples.iloc[idx]['text']
        label = self.samples.iloc[idx]['label']
        return txt, label

In [9]:
trainer = SampleLoader(annotation_file='train.csv')
trainloader = DataLoader(dataset=trainer, batch_size=128, shuffle=False)

In [11]:
for i , (text, label) in enumerate(trainloader):
    print(text)
    print(label)
    break

("Ciao, Scusate il ritardo, ho appena risposto ai ticket! Ciao, Andrea -- Andrea Di Pasquale Software Developer Sent from my mobile.   Da : Bruno Muschitiello Inviato : Thursday, December 11, 2014 10:16 AM A : Andrea Di Pasquale Cc : Cristian Vardaro Oggetto : Fwd: [!AIJ-631-57916]: Don't worked offline installation RCS 9.5   Ciao Andrea,  hanno risposto alla richiesta che avevi fatto riguardo il CD offline. Domani puoi continuare a seguire la questione? Oppure se vuoi che chieda loro qualcosa o che gli comunichi qualcosa fammi sapere. Grazie Bruno -------- Messaggio originale -------- Oggetto: [!AIJ-631-57916]: Don't worked offline installation RCS 9.5 Data: Thu, 11 Dec 2014 01:18:00 -0500 Mittente: i.eugene <support@hackingteam.com> Rispondi-a: <support@hackingteam.com> A: <rcs-support@hackingteam.com> i.eugene updated #AIJ-631-57916 ------------------------------- Don't worked offline installation RCS 9.5 ----------------------------------------- Ticket ID: AIJ-631-57916 URL: Name: 

In [15]:
import pandas as pd

# Load the datasets
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
val_data = pd.read_csv('validation.csv')

# Inspect the data
print(train_data.head())
print(train_data.columns)

                                                text  label
0  Already, the mayor of Green Bay is having ribb...      1
1  I argued for years that we need to move from a...      1
2  The video doesn't show the broader picture of ...      0
3  A ham and cheese sandwich on one slice of brea...      0
4  He has been able to attract independents and D...      1
Index(['text', 'label'], dtype='object')


In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense

# Load the datasets
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
val_data = pd.read_csv('validation.csv')

# Define constants
MAX_NUM_WORDS = 10000  # Use the top 10,000 most frequent words
MAX_SEQUENCE_LENGTH = 200  # Maximum length of sequences

# Tokenize the text data
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(train_data['text'].values)

# Convert text to sequences
X_train = tokenizer.texts_to_sequences(train_data['text'].values)
X_val = tokenizer.texts_to_sequences(val_data['text'].values)
X_test = tokenizer.texts_to_sequences(test_data['text'].values)

# Pad sequences to ensure uniform input size
X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH)
X_val = pad_sequences(X_val, maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)

# Convert labels to numpy arrays
y_train = train_data['label'].values
y_val = val_data['label'].values
y_test = test_data['label'].values


model = Sequential()
model.add(Embedding(input_dim=MAX_NUM_WORDS, output_dim=128, input_length=MAX_SEQUENCE_LENGTH))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=5))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy*100:.2f}%')

C:\Users\Sina\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ ?                      │   0 (unbuilt) │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
382/382 ━━━━━━━━━━━━━━━━━━━━ 14s 31ms/step - accuracy: 0.8478 - loss: 0.2914 - val_accuracy: 0.9751 - val_loss: 0.0649
Epoch 2/10
382/382 ━━━━━━━━━━━━━━━━━━━━ 12s 31ms/step - accuracy: 0.9900 - loss: 0.0313 - val_accuracy: 0.9764 - val_loss: 0.0662
Epoch 3/10
382/382 ━━━━━━━━━━━━━━━━━━━━ 12s 32ms/step - accuracy: 0.9972 - loss: 0.0118 - val_accuracy: 0.9758 - val_loss: 0.0642
Epoch 4/10
382/382 ━━━━━━━━━━━━━━━━━━━━ 12s 31ms/step - accuracy: 0.9973 - loss: 0.0088 - val_accuracy: 0.9751 - val_loss: 0.0967
Epoch 5/10
382/382 ━━━━━━━━━━━━━━━━━━━━ 12s 32ms/step - accuracy: 0.9982 - loss: 0.0060 - val_accuracy: 0.9771 - val_loss: 0.0747
Epoch 6/10
382/382 ━━━━━━━━━━━━━━━━━━━━ 12s 31ms/step - accuracy: 0.9982 - loss: 0.0054 - val_accuracy: 0.9705 - val_loss: 0.1971
Epoch 7/10
382/382 ━━━━━━━━━━━━━━━━━━━━ 12s 32ms/step - accuracy: 0.9973 - loss: 0.0066 - val_accuracy: 0.9699 - val_loss: 0.1246
Epoch 8/10
382/382 ━━━━━━━━━━━━━━━━━━━━ 13s 33ms/step - accuracy: 0.9977 - loss: 0.0059 - 

In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense, Dropout
import numpy as np

# Load the datasets
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
val_data = pd.read_csv('validation.csv')

# Define constants
MAX_NUM_WORDS = 10000  # Use the top 10,000 most frequent words
MAX_SEQUENCE_LENGTH = 200  # Maximum length of sequences

# Tokenize the text data
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(train_data['text'].values)

# Convert text to sequences
X_train = tokenizer.texts_to_sequences(train_data['text'].values)
X_val = tokenizer.texts_to_sequences(val_data['text'].values)
X_test = tokenizer.texts_to_sequences(test_data['text'].values)

# Pad sequences to ensure uniform input size
X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH)
X_val = pad_sequences(X_val, maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)

# Convert labels to numpy arrays
y_train = train_data['label'].values
y_val = val_data['label'].values
y_test = test_data['label'].values

# Load GloVe embeddings
def load_glove_embeddings(glove_file, word_index, embedding_dim=100):
    embeddings_index = {}
    with open(glove_file, encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            
    return embedding_matrix

# Path to the GloVe file
glove_file = 'glove.6B.100d.txt'  # Change this to the actual path of the GloVe file
embedding_dim = 100
word_index = tokenizer.word_index
embedding_matrix = load_glove_embeddings(glove_file, word_index, embedding_dim)

# Define the CNN model with pre-trained embeddings and dropout layers
model = Sequential()
model.add(Embedding(input_dim=len(word_index) + 1,
                    output_dim=embedding_dim,
                    weights=[embedding_matrix],
                    input_length=MAX_SEQUENCE_LENGTH,
                    trainable=False))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=5))
model.add(Dropout(0.5))  # Add dropout layer
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Add dropout layer
model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy*100:.2f}%')


C:\Users\Sina\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │     1,228,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ ?                      │   0 (unbuilt) │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,228,400 (4.69 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 1,228,400 (4.69 MB)

Epoch 1/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.6274 - loss: 0.6657 - val_accuracy: 0.6464 - val_loss: 0.6366
Epoch 2/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - accuracy: 0.6504 - loss: 0.6368 - val_accuracy: 0.6440 - val_loss: 0.6294
Epoch 3/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.6503 - loss: 0.6258 - val_accuracy: 0.6448 - val_loss: 0.6280
Epoch 4/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - accuracy: 0.6557 - loss: 0.6144 - val_accuracy: 0.6488 - val_loss: 0.6251
Epoch 5/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.6604 - loss: 0.6092 - val_accuracy: 0.6288 - val_loss: 0.6286
Epoch 6/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.6810 - loss: 0.5892 - val_accuracy: 0.6376 - val_loss: 0.6203
Epoch 7/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - accuracy: 0.6978 - loss: 0.5731 - val_accuracy: 0.6424 - val_loss: 0.6206
Epoch 8/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.7110 - loss: 0.5501 - val_accu

In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Bidirectional, GlobalMaxPooling1D, Dense, Dropout
import numpy as np
from sklearn.model_selection import KFold

# Load the datasets
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
val_data = pd.read_csv('validation.csv')

# Combine train and validation data for cross-validation
combined_data = pd.concat([train_data, val_data])

# Define constants
MAX_NUM_WORDS = 10000  # Use the top 10,000 most frequent words
MAX_SEQUENCE_LENGTH = 200  # Maximum length of sequences

# Tokenize the text data
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(combined_data['text'].values)

# Convert text to sequences
X = tokenizer.texts_to_sequences(combined_data['text'].values)
X_test = tokenizer.texts_to_sequences(test_data['text'].values)

# Pad sequences to ensure uniform input size
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)

# Convert labels to numpy arrays
y = combined_data['label'].values
y_test = test_data['label'].values

# Load GloVe embeddings
def load_glove_embeddings(glove_file, word_index, embedding_dim=100):
    embeddings_index = {}
    with open(glove_file, encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            
    return embedding_matrix

# Path to the GloVe file
glove_file = 'glove.6B.100d.txt'  # Change this to the actual path of the GloVe file
embedding_dim = 100
word_index = tokenizer.word_index
embedding_matrix = load_glove_embeddings(glove_file, word_index, embedding_dim)

# Function to create the Bidirectional LSTM model
def create_bilstm_model():
    model = Sequential()
    model.add(Embedding(input_dim=len(word_index) + 1,
                        output_dim=embedding_dim,
                        weights=[embedding_matrix],
                        input_length=MAX_SEQUENCE_LENGTH,
                        trainable=False))
    model.add(Bidirectional(LSTM(units=128, return_sequences=True)))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Perform K-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
accuracies = []

for train_index, val_index in kf.split(X):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    model = create_bilstm_model()
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))
    
    loss, accuracy = model.evaluate(X_val, y_val)
    accuracies.append(accuracy)

average_accuracy = np.mean(accuracies)
print(f'Average Cross-Validation Accuracy: {average_accuracy*100:.2f}%')

# Evaluate the final model on the test set
model = create_bilstm_model()
model.fit(X, y, epochs=10, batch_size=32)
loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy*100:.2f}%')


C:\Users\Sina\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
344/344 ━━━━━━━━━━━━━━━━━━━━ 54s 145ms/step - accuracy: 0.8541 - loss: 0.3273 - val_accuracy: 0.9440 - val_loss: 0.1346
Epoch 2/10
344/344 ━━━━━━━━━━━━━━━━━━━━ 51s 149ms/step - accuracy: 0.9535 - loss: 0.1145 - val_accuracy: 0.9644 - val_loss: 0.0940
Epoch 3/10
344/344 ━━━━━━━━━━━━━━━━━━━━ 53s 155ms/step - accuracy: 0.9756 - loss: 0.0666 - val_accuracy: 0.9713 - val_loss: 0.0756
Epoch 4/10
344/344 ━━━━━━━━━━━━━━━━━━━━ 56s 163ms/step - accuracy: 0.9846 - loss: 0.0420 - val_accuracy: 0.9749 - val_loss: 0.0720
Epoch 5/10
344/344 ━━━━━━━━━━━━━━━━━━━━ 60s 176ms/step - accuracy: 0.9921 - loss: 0.0237 - val_accuracy: 0.9614 - val_loss: 0.1166
Epoch 6/10
344/344 ━━━━━━━━━━━━━━━━━━━━ 60s 176ms/step - accuracy: 0.9945 - loss: 0.0194 - val_accuracy: 0.9756 - val_loss: 0.0779
Epoch 7/10
344/344 ━━━━━━━━━━━━━━━━━━━━ 60s 175ms/step - accuracy: 0.9966 - loss: 0.0122 - val_accuracy: 0.9807 - val_loss: 0.0706
Epoch 8/10
344/344 ━━━━━━━━━━━━━━━━━━━━ 60s 175ms/step - accuracy: 0.9991 - loss: 0

In [2]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam

# Load the datasets
train_data = pd.read_csv('train.csv')
val_data = pd.read_csv('validation.csv')
test_data = pd.read_csv('test.csv')

# Define constants
MAX_NUM_WORDS = 10000  # Use the top 10,000 most frequent words
MAX_SEQUENCE_LENGTH = 200  # Maximum length of sequences

# Tokenize the text data
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(train_data['text'].values)

# Convert text to sequences
X_train = tokenizer.texts_to_sequences(train_data['text'].values)
X_val = tokenizer.texts_to_sequences(val_data['text'].values)
X_test = tokenizer.texts_to_sequences(test_data['text'].values)

# Pad sequences to ensure uniform input size
X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH)
X_val = pad_sequences(X_val, maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)

# Convert labels to numpy arrays
y_train = train_data['label'].values
y_val = val_data['label'].values
y_test = test_data['label'].values

# Define the RNN model
model = Sequential()
model.add(Embedding(input_dim=MAX_NUM_WORDS, output_dim=128, input_length=MAX_SEQUENCE_LENGTH))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification

model.compile(optimizer=Adam(learning_rate=1e-3), loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy*100:.2f}%')

C:\Users\Sina\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
382/382 ━━━━━━━━━━━━━━━━━━━━ 89s 223ms/step - accuracy: 0.8903 - loss: 0.2453 - val_accuracy: 0.9646 - val_loss: 0.0919
Epoch 2/10
382/382 ━━━━━━━━━━━━━━━━━━━━ 91s 238ms/step - accuracy: 0.9762 - loss: 0.0736 - val_accuracy: 0.9666 - val_loss: 0.1138
Epoch 3/10
382/382 ━━━━━━━━━━━━━━━━━━━━ 103s 269ms/step - accuracy: 0.9916 - loss: 0.0297 - val_accuracy: 0.9587 - val_loss: 0.1565
Epoch 4/10
382/382 ━━━━━━━━━━━━━━━━━━━━ 106s 276ms/step - accuracy: 0.9914 - loss: 0.0317 - val_accuracy: 0.9718 - val_loss: 0.1214
Epoch 5/10
382/382 ━━━━━━━━━━━━━━━━━━━━ 100s 261ms/step - accuracy: 0.9926 - loss: 0.0232 - val_accuracy: 0.9646 - val_loss: 0.1310
Epoch 6/10
382/382 ━━━━━━━━━━━━━━━━━━━━ 97s 254ms/step - accuracy: 0.9985 - loss: 0.0063 - val_accuracy: 0.9627 - val_loss: 0.2143
Epoch 7/10
382/382 ━━━━━━━━━━━━━━━━━━━━ 100s 262ms/step - accuracy: 0.9978 - loss: 0.0100 - val_accuracy: 0.9725 - val_loss: 0.1497
Epoch 8/10
382/382 ━━━━━━━━━━━━━━━━━━━━ 98s 256ms/step - accuracy: 0.9993 - los